In [1]:
from __future__ import (division, unicode_literals, absolute_import,
                        print_function)

from ema_workbench import (Model, RealParameter, Constant, CategoricalParameter, 
                           TimeSeriesOutcome, ScalarOutcome, ema_logging, perform_experiments, 
                           MultiprocessingEvaluator, Policy)
from ema_workbench.connectors.vensim import (VensimModel , VensimModelStructureInterface, set_value)
from ema_workbench.em_framework.samplers import sample_levers
from ema_workbench.util.utilities import save_results
from ema_workbench import load_results
from ema_workbench.analysis.pairs_plotting import (pairs_lines, pairs_scatter)
from ema_workbench.analysis import clusterer, plotting, Density, prim
from ema_workbench.analysis.plotting import lines, envelopes, kde_over_time, multiple_densities



import timeit
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import itertools

from ema_workbench.analysis.plotting import lines

C:\Programs\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:27: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath

# Select vensim file

In [2]:
wd = r'./VENSIM' # Directory of the vensim model
model_file=r'Maasmodel_v075.vpmx' # Name of the vensim model

# General manual input

In [3]:
#FILL IN THIS CELL
#WEIR
Crest_level_weir1 = 8.35
Crest_level_weir2 = 7.15
Crest_level_weir3 = 6.9
Crest_level_weir4 = 7.2
Crest_level_weir5 = [8.4, 8.6]
Crest_level_weir6 = 10.9
Location_weir1    = 57500
Location_weir2    = 69500
Location_weir3    = 87500
Location_weir4    = 135500
Location_weir5    = [163500, 150500]
Location_weir6    = 189500
Leakage_low_weir1    = 1.2e6
Leakage_low_weir2    = 1e6
Leakage_low_weir3    = 1e6
Leakage_low_weir4    = 3e6
Leakage_low_weir5    = 3e6
Leakage_low_weir6    = 2e6
Leakage_up_weir1     = 6e6
Leakage_up_weir2     = 5e6
Leakage_up_weir3     = 5e6
Leakage_up_weir4     = 9e6
Leakage_up_weir5     = 9e6
Leakage_up_weir6     = 8e6

#LOCK
Intensity_commercial_lock1 = 44
Intensity_commercial_lock2 = 62
Intensity_commercial_lock3 = 439
Intensity_commercial_lock4 = 547
Intensity_commercial_lock5 = 289
Intensity_commercial_lock6 = 334
Intensity_leisure_lock1 = 643
Intensity_leisure_lock2 = 518
Intensity_leisure_lock3 = 458
Intensity_leisure_lock4 = 498
Intensity_leisure_lock5 = 511
Intensity_leisure_lock6 = 807

Make arrays to feed lookups

In [4]:
#NAME CLIMATE SCENARIOS
CS1 = 'Climate scenario GL'
CS2 = 'Climate scenario WH'

#SAVE FIGURE
save = False
fig_size = (10,7)
dpi = 300
%matplotlib inline

#Starting year
start_year = 2020
end_year   = 2100


#100 years
years = np.linspace(start_year,end_year,81)

#SECONDS TO Week
SEC_WEEK = 60*60*24*7 

#Different draught durations in weeks
Draught_durations = [4,6,8,10,12,14,16] #Duur van de droogte periode variërend van 4 tot 16 weken

#Different draught intensities in m3/s
Draught_intensities = [20,25,30,35,40,45,1000] #Maximale afvoer in periodes van droogte, 1000 staat voor geen droogte periode 

#Distance from inflow
distance_inflow = np.linspace(500, 189500, 190)

#100 year discharge series
year_100_dischargeseries = list(range(1911,2015))

# Model

In [5]:
# instantiate a model
vensimModel = VensimModel('MaasModel', wd=wd, model_file=model_file)


# Experiments info and load the results

In [6]:
policies = [Policy('No policy',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Crest Grave raised',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.6, 'Levelling restrictions switch': 0, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Dynamic crest level',**{'Crest level summer': 0.2, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Pumps on',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 1, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Levelling restrictions',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 1, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Relocate weir',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 0, 'Weir location[Weir5]': 150500, 'Crest level weir[Weir6]':8.5}),
Policy('Relocate weir and raise crest level Lith',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 0, 'Weir location[Weir5]': 150500, 'Crest level weir[Weir6]':8.7}),
Policy('Crest level raised and pumps on',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.6, 'Levelling restrictions switch': 0, 'Pump switch': 1, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Dynamic crest level and pumps on',**{'Crest level summer': 0.2, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 1, 'Weir location[Weir6]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Crest level raised and levelling restrictions',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.6, 'Levelling restrictions switch': 1, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Dynamic crest level and levelling restrictions',**{'Crest level summer': 0.2, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 1, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Relocate weir and pumps on',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 1, 'Weir location[Weir5]': 150500, 'Crest level weir[Weir6]':8.5}),
Policy('Relocate weir and levelling restrictions',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 1, 'Pump switch': 0, 'Weir location[Weir5]': 150500, 'Crest level weir[Weir6]':8.5})
           ]


In [7]:
nr_scenarios = 8000
nr_policies = len(policies)
nr_experiments = nr_scenarios*nr_policies


print('Simulating the model with', nr_scenarios, 'scenarios and', nr_policies, 'design decisions')

Simulating the model with 8000 scenarios and 13 design decisions


Load results

In [8]:
data = r'C:\Users\douwe\OneDrive\Documenten\1.0 School\8.0 Thesis\5.0 Execution\Maasmodel\Results\Policies_104000a.tar'
results = load_results(data)
experiments, outcomes = results

# Data Postprocessing

In [9]:
keys = list(outcomes.keys())
keys

['TIME',
 'Percentage of full fresh water buffer[Weir1]',
 'Percentage of full fresh water buffer[Weir2]',
 'Percentage of full fresh water buffer[Weir3]',
 'Percentage of full fresh water buffer[Weir4]',
 'Percentage of full fresh water buffer[Weir5]',
 'Percentage of full fresh water buffer[Weir6]',
 'Average velocity through weir section[Weir1]',
 'Average velocity through weir section[Weir2]',
 'Average velocity through weir section[Weir3]',
 'Average velocity through weir section[Weir4]',
 'Average velocity through weir section[Weir5]',
 'Average velocity through weir section[Weir6]',
 '"I/C value at lock"[Weir1]',
 '"I/C value at lock"[Weir2]',
 '"I/C value at lock"[Weir3]',
 '"I/C value at lock"[Weir4]',
 '"I/C value at lock"[Weir5]',
 '"I/C value at lock"[Weir6]',
 'Days of free flow',
 'Water depth[S1]',
 'Water depth[S2]',
 'Water depth[S3]',
 'Water depth[S4]',
 'Water depth[S5]',
 'Water depth[S6]',
 'Water depth[S7]',
 'Water depth[S8]',
 'Water depth[S9]',
 'Water depth[S

In [10]:
#Delete NAN's from crest level weir Lith in experiments
del experiments['Crest level weir[Wei6]']
experiments['Crest level weir[Weir6]'].fillna(8.5,inplace=True)

Create DataFrame containing maximum water depths

In [37]:
depth= keys[20:210]
df_depth = pd.DataFrame([[51]]*len(outcomes['TIME']))   

for m in range(len(depth)):
    array2 = []
    for n in range(len(outcomes['TIME'])): 
        array2.append(np.max(outcomes[depth[m]][n]))    
    df_depth[depth[m]] = array2  

In [48]:
#Add uncertainties to depth outcomes dataframe 
uncertain = experiments.keys().tolist()

for i in range(len(uncertain)):
    if experiments.keys()[i]==uncertain[i]:
        a = experiments.iloc[:,i].tolist()
        df_depth[uncertain[i]] = a

In [50]:
#Save dataframe to excel 
df_depth.to_excel(r"C:\Users\douwe\OneDrive\Documenten\1.0 School\8.0 Thesis\5.0 Execution\Maasmodel\Results\policy_depths2_max.xlsx")

Create DataFrame containing minimum water depths

In [11]:
#Make dataframe with outcomes
fwb = ['Percentage of full fresh water buffer[Weir1]',
 'Percentage of full fresh water buffer[Weir2]',
 'Percentage of full fresh water buffer[Weir3]',
 'Percentage of full fresh water buffer[Weir4]',
 'Percentage of full fresh water buffer[Weir5]',
 'Percentage of full fresh water buffer[Weir6]','Days of free flow']


df_index = pd.DataFrame([[51]]*len(outcomes['TIME']))      

        
for j in range(len(fwb)):
    array = []
    for i in range(len(outcomes['TIME'])): 
        array.append(outcomes[fwb[j]][i].argmin())
    df_index[fwb[j]] = array


In [12]:
#Make water depth dataframe with water depth outcomes
depth= keys[20:210]

df_depth = pd.DataFrame([[51]]*len(outcomes['TIME']))        
        
for j in range(len(depth)):
    array = []
    for i in range(len(outcomes['TIME'])): 
        if j<59:
            array.append(outcomes[depth[j]][i][df_index.iloc[i,1]])
        elif j>58 and j<71:
            array.append(outcomes[depth[j]][i][df_index.iloc[i,2]])
        elif j>70 and j<89:
            array.append(outcomes[depth[j]][i][df_index.iloc[i,3]])
        elif j>88 and j<138:
            array.append(outcomes[depth[j]][i][df_index.iloc[i,4]])
        elif j>137 and j<165:
            array.append(outcomes[depth[j]][i][df_index.iloc[i,5]])
        elif j>164 and j<=190:
            array.append(outcomes[depth[j]][i][df_index.iloc[i,6]])
    df_depth[depth[j]] = array



In [ ]:
depth= keys[20:210]

df_depth = pd.DataFrame([[51]]*len(outcomes['TIME']))        
        
for j in range(len(depth)):
    array = []
    for i in range(len(outcomes['TIME'])): 
        array.append(np.max(outcomes[depth[j]][i]))
    df_depth[depth[j]] = array


In [ ]:
df_depth

In [14]:

#Add uncertainties to depth outcomes dataframe 
uncertain = experiments.keys().tolist()

for i in range(len(uncertain)):
    if experiments.keys()[i]==uncertain[i]:
        a = experiments.iloc[:,i].tolist()
        df_depth[uncertain[i]] = a
        


In [16]:
#Save dataframe to excel 
df_depth.to_excel(r"C:\Users\douwe\OneDrive\Documenten\1.0 School\8.0 Thesis\5.0 Execution\Maasmodel\Results\policy_depths2_basedonfwb.xlsx")

In [28]:

#Make dataframe with outcomes
fwb = ['Percentage of full fresh water buffer[Weir1]',
 'Percentage of full fresh water buffer[Weir2]',
 'Percentage of full fresh water buffer[Weir3]',
 'Percentage of full fresh water buffer[Weir4]',
 'Percentage of full fresh water buffer[Weir5]',
 'Percentage of full fresh water buffer[Weir6]','Days of free flow']
vel = keys[7:13]
ship= keys[13:19]


df_outcome = pd.DataFrame([[51]]*len(outcomes['TIME']))        
        
for j in range(len(fwb)):
    array = []
    for i in range(len(outcomes['TIME'])): 
        array.append(outcomes[fwb[j]][i][-1])
    df_outcome[fwb[j]] = array

for k in range(len(vel)):
    array1 = []
    for l in range(len(outcomes['TIME'])): 
        array1.append(np.mean(outcomes[vel[k]][l]))    
    df_outcome[vel[k]] = array1

for m in range(len(ship)):
    array2 = []
    for n in range(len(outcomes['TIME'])): 
        array2.append(np.max(outcomes[ship[m]][n]))    
    df_outcome[ship[m]] = array2  

del df_outcome[0]



In [29]:

#Add uncertainties to outcomes dataframe 
uncertain = experiments.keys().tolist()

for i in range(len(uncertain)):
    if experiments.keys()[i]==uncertain[i]:
        a = experiments.iloc[:,i].tolist()
        df_outcome[uncertain[i]] = a
        


In [30]:

#Rename outcome columns
df_outcome.rename(columns ={'Percentage of full fresh water buffer[Weir1]': 'Percentage of full fresh water buffer weir section Linne',
                           'Percentage of full fresh water buffer[Weir2]': 'Percentage of full fresh water buffer weir section Roermond',
                           'Percentage of full fresh water buffer[Weir3]': 'Percentage of full fresh water buffer weir section Belfeld',
                           'Percentage of full fresh water buffer[Weir4]': 'Percentage of full fresh water buffer weir section Sambeek',
                           'Percentage of full fresh water buffer[Weir5]': 'Percentage of full fresh water buffer weir section Grave',
                           'Percentage of full fresh water buffer[Weir6]': 'Percentage of full fresh water buffer weir section Lith'}, inplace = True)
                           


In [31]:
Save dataframe to excel 
df_outcome.to_excel(r"C:\Users\douwe\OneDrive\Documenten\1.0 School\8.0 Thesis\5.0 Execution\Maasmodel\Results\policy_dataframe.xlsx")